In [1]:
import os
import numpy as np
from spectral import envi
import json

with open("train_only_labeled.json", "r") as f:
    data = json.load(f)

annotations = data["annotations"]
records = data["records"]
cameras = data["cameras"]
vis_wls = cameras[0]["wavelengths"]
nir_wls = cameras[1]["wavelengths"]

ripeness_mapping = {"unripe": 0, "perfect": 1, "overripe": 2}

X = []
Y = []

for annotation in annotations:
    record = next((r for r in records if r["id"] == annotation["record_id"]), None)
    if record:
        try:
            image = envi.open(
                f"data/{record["files"]["header_file"]}",
                image=f"data/{record["files"]["data_file"]}",
            )
            X.append({"image": image.load(), "camera": record["camera_type"]})
            Y.append(ripeness_mapping[annotation["ripeness_state"]])
        except Exception as e:
            print(f"Error loading image: {e}")

X = np.array(X, dtype=object)
Y = np.array(Y, dtype=int)

print(f"Loaded dataset: {len(X)} samples.")

Error loading image: Unable to locate file "data/Kiwi/NIR/day_01/kiwi_day_01_03_front.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
Error loading image: Unable to locate file "data/Kiwi/VIS/day_01/kiwi_day_01_03_front.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
Error loading image: Unable to locate file "data/Kiwi/NIR/day_01/kiwi_day_01_03_back.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
Error loading image: Unable to locate file "data/Kiwi/VIS/day_01/kiwi_day_01_03_back.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
Error loading image: Unable to locate file "data/Kiwi/NIR/day_01/kiwi_day_01_28_front.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
Error loading image: U

In [8]:
import numpy as np


def fourier_features(image):
    """Compute Fourier Transform magnitudes for each pixel's spectral vector."""
    fft_magnitude = np.abs(
        np.fft.fft(image, axis=-1)
    )  # Apply FFT along spectral channels
    return fft_magnitude[:, :, : image.shape[-1] // 2]  # Use only positive frequencies


def fourier_2d_features(image):
    """Compute 2D Fourier Transform for each spectral channel."""
    fft_features = []
    for i in range(image.shape[-1]):  # Iterate through spectral channels
        channel_fft = np.fft.fft2(image[:, :, i])
        fft_magnitude = np.abs(channel_fft)
        fft_features.append(
            fft_magnitude.flatten()
        )  # Flatten to create a feature vector
    return np.concatenate(fft_features)  # Combine all channels


def low_freq_features(fft_magnitude, k=10):
    """Select the first k low-frequency components from FFT features."""
    return fft_magnitude[:, :, :k].flatten()

In [9]:
X1 = []
Y1 = []

for x, y in zip(X, Y):
    if x["camera"] == "VIS":
        X1.append(x)
        Y1.append(y)

X1 = np.array(X1, dtype=object)
Y1 = np.array(Y1, dtype=int)

print(f"Loaded dataset: {len(X1)} samples.")

Loaded dataset: 92 samples.


In [10]:
def pad_features(features, fill_value=0):
    """Pad feature vectors to have the same length."""
    max_length = max(len(f) for f in features)
    return np.array(
        [
            np.pad(f, (0, max_length - len(f)), constant_values=fill_value)
            for f in features
        ]
    )


X_padded = [pad_features(image["image"]) for image in X1]

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Example: Process a list of spectral images
X_fft = [fourier_2d_features(image) for image in X_padded]  # Fourier features
y = Y1  # Labels: ripe, unripe, perfect

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_fft, y, test_size=0.2, random_state=42
)

# Classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [6]:
import os
import numpy as np
from spectral import envi
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load JSON data
with open("train_only_labeled.json", "r") as f:
    data = json.load(f)

annotations = data["annotations"]
records = data["records"]
cameras = data["cameras"]
vis_wls = cameras[0]["wavelengths"]
nir_wls = cameras[1]["wavelengths"]

ripeness_mapping = {"unripe": 0, "perfect": 1, "overripe": 2}

X = []
Y = []

# Load hyperspectral images
for annotation in annotations:
    record = next((r for r in records if r["id"] == annotation["record_id"]), None)
    if record:
        header_file = os.path.join("data", record["files"]["header_file"])
        data_file = os.path.join("data", record["files"]["data_file"])
        if os.path.exists(header_file) and os.path.exists(data_file):
            try:
                image = envi.open(header_file, image=data_file)
                X.append({"image": image.load(), "camera": record["camera_type"]})
                Y.append(ripeness_mapping[annotation["ripeness_state"]])
            except Exception as e:
                print(f"Error loading image: {e}")
        else:
            print(f"Missing file: {header_file} or {data_file}")

X = np.array(X, dtype=object)
Y = np.array(Y, dtype=int)
print(f"Loaded dataset: {len(X)} samples.")


# Fourier Features Functions
def fourier_features(image):
    """Compute Fourier Transform magnitudes for each pixel's spectral vector."""
    fft_magnitude = np.abs(np.fft.fft(image, axis=-1))  # FFT along spectral axis
    return fft_magnitude[:, :, : image.shape[-1] // 2]  # Use positive frequencies


def fourier_2d_features(image):
    """Compute 2D Fourier Transform for each spectral channel."""
    fft_features = []
    for i in range(image.shape[-1]):
        channel_fft = np.fft.fft2(image[:, :, i])
        fft_magnitude = np.abs(channel_fft)
        fft_features.append(fft_magnitude.flatten())
    return np.concatenate(fft_features)


def pad_features(features, fill_value=0):
    """Pad feature vectors to have the same length."""
    max_length = max(len(f) for f in features)
    return np.array(
        [
            np.pad(f, (0, max_length - len(f)), constant_values=fill_value)
            for f in features
        ]
    )


# Filter only VIS images
X_vis = []
Y_vis = []

for x, y in zip(X, Y):
    if x["camera"] == "VIS":
        X_vis.append(x["image"])
        Y_vis.append(y)

X_vis = np.array(X_vis, dtype=object)
Y_vis = np.array(Y_vis, dtype=int)
print(f"Filtered VIS dataset: {len(X_vis)} samples.")

# Extract Fourier Features
X_fft = []
for image in X_vis:
    fft_features = fourier_2d_features(image)  # Flattened 2D FFT features
    X_fft.append(fft_features)

Missing file: data/Kiwi/NIR/day_01/kiwi_day_01_03_front.hdr or data/Kiwi/NIR/day_01/kiwi_day_01_03_front.bin
Missing file: data/Kiwi/VIS/day_01/kiwi_day_01_03_front.hdr or data/Kiwi/VIS/day_01/kiwi_day_01_03_front.bin
Missing file: data/Kiwi/NIR/day_01/kiwi_day_01_03_back.hdr or data/Kiwi/NIR/day_01/kiwi_day_01_03_back.bin
Missing file: data/Kiwi/VIS/day_01/kiwi_day_01_03_back.hdr or data/Kiwi/VIS/day_01/kiwi_day_01_03_back.bin
Missing file: data/Kiwi/NIR/day_01/kiwi_day_01_28_front.hdr or data/Kiwi/NIR/day_01/kiwi_day_01_28_front.bin
Missing file: data/Kiwi/VIS/day_01/kiwi_day_01_28_front.hdr or data/Kiwi/VIS/day_01/kiwi_day_01_28_front.bin
Missing file: data/Kiwi/NIR/day_01/kiwi_day_01_28_back.hdr or data/Kiwi/NIR/day_01/kiwi_day_01_28_back.bin
Missing file: data/Kiwi/VIS/day_01/kiwi_day_01_28_back.hdr or data/Kiwi/VIS/day_01/kiwi_day_01_28_back.bin
Missing file: data/Kiwi/NIR/day_02/kiwi_day_02_10_front.hdr or data/Kiwi/NIR/day_02/kiwi_day_02_10_front.bin
Missing file: data/Kiwi/VIS

In [ ]:
X_fft = [fourier_2d_features(image) for image in X_vis]
X_fft_padded = pad_features(X_fft)
Y_vis = np.array(Y_vis)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Balance the dataset
ros = RandomOverSampler(random_state=42)
X_resampled, Y_resampled = ros.fit_resample(X_fft_padded, Y_vis)

print("Original class distribution:", Counter(Y_vis))
print("Resampled class distribution:", Counter(Y_resampled))

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.2, random_state=42
)

# Train RandomForest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and Evaluate
y_pred = clf.predict(X_test)
print(y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
from collections import Counter

print("Class distribution in VIS dataset:", Counter(Y_vis))

Class distribution in VIS dataset: Counter({np.int64(1): 50, np.int64(2): 26, np.int64(0): 16})
